In [2]:
from adaptive_consistency import AC, BetaStoppingCriteria

ac = AC(stop_criteria=BetaStoppingCriteria(0.95), max_gens = 40)

In [5]:
import pandas as pd
df = pd.read_csv('../data/final.csv')
df.head()

,Name,Category,Question,Correct Answer,CoT_0,Final Answer_0,Instruction Violation_0,CoT_1,Final Answer_1,Instruction Violation_1,...,Final Answer_37,Instruction Violation_37,CoT_38,Final Answer_38,Instruction Violation_38,CoT_39,Final Answer_39,Instruction Violation_39,Difficulty,Model
0,GSM8K_hard,Challenging Math,Janet’s ducks lay 16 eggs per day. She eats th...,-9867630.0,Step 1: Calculate the total number of eggs lai...,0.0,"[(0, 0)]",Step 1: Janet's ducks lay 16 eggs per day.\nSt...,0.0,"[(0, 0)]",...,18.0,"[(0, 0)]",Step 1: Determine the total number of eggs lai...,0.0,"[(0, 0)]",Step 1: Calculate the total number of eggs lai...,18.0,"[(0, 0)]",hard,claude-3-haiku-20240307
1,GSM8K_hard,Challenging Math,A robe takes 2287720 bolts of blue fiber and h...,3431580.0,Step 1: The robe takes 2287720 bolts of blue f...,3431580.0,"[(0, 0)]",Step 1: The robe takes 2287720 bolts of blue f...,3431580.0,"[(0, 0)]",...,3431580.0,"[(0, 0)]",Step 1: The question states that the robe take...,3431580.0,"[(0, 0)]",Step 1: The question states that a robe takes ...,3431580.0,"[(0, 0)]",hard,claude-3-haiku-20240307
2,GSM8K_hard,Challenging Math,James decides to run 1793815 sprints 1793815 t...,322886700.0,Step 1: Identify the given information.\n ...,193.0,"[(0, 0)]",Step 1: The problem states that James decides ...,107.0,"[(0, 0)]",...,192.0,"[(0, 0)]",Step 1: Understand the given information.\n- J...,193.0,"[(0, 0)]",Step 1: Understand the given information.\n St...,193.0,"[(0, 0)]",hard,claude-3-haiku-20240307
3,GSM8K_hard,Challenging Math,"Every day, Wendi feeds each of her chickens th...",-6887448.0,Step 1: Determine the total number of cups of ...,error,"[(0, 0)]",Step 1: Determine the total number of cups of ...,2117.5,"[(0, 0)]",...,-6887448.0,"[(0, 0)]",Step 1: Understand the given information.\n- W...,-6.0,"[(0, 0)]",Step 1: Determine the total amount of feed Wen...,error,"[(0, 0)]",hard,claude-3-haiku-20240307
4,GSM8K_hard,Challenging Math,Kylar went to the store to buy glasses for his...,21459061.0,Step 1: Determine the price of each glass.\n ...,21459061.0,"[(0, 0)]",Step 1: Determine the price of each glass.\n ...,21.0,"[(0, 0)]",...,21.0,"[(0, 0)]",Step 1: Determine the cost of each glass.\n ...,21.0,"[(0, 0)]",Step 1: Determine the cost of each glass.\n ...,21.0,"[(0, 0)]",hard,claude-3-haiku-20240307


In [16]:
values_list = [df[f'Final Answer_{i}'].iloc[300] for i in range(40)]
answers = []
for i in range(40):
    answers.append(values_list[i]) # Example openai.Completion.create
    if ac.should_stop(answers):
        print(i)
        break
print(len(answers))

3
4


In [17]:
import pandas as pd
from collections import Counter

# Assuming df is your DataFrame and it has columns Final Answer_0 to Final Answer_39

def majority_vote(answers):
    """ Return the most common element in the answers list. """
    if answers:
        return Counter(answers).most_common(1)[0][0]
    return None

# Initialize new columns
df['asc_steps'] = 0
df['asc_answer'] = None

# Iterate over each row of the DataFrame
for index, row in df.iterrows():
    answers = []
    for i in range(40):
        answer = row[f'Final Answer_{i}']
        answers.append(answer)
        if ac.should_stop(answers):
            break
    # Save the length of answers after the loop
    df.at[index, 'asc_steps'] = len(answers)
    # Save the majority vote of answers
    df.at[index, 'asc_answer'] = majority_vote(answers)

# Example print to show the new DataFrame structure
print(df[['asc_steps', 'asc_answer']])

      asc_steps asc_answer
0            18        0.0
1             4  3431580.0
2            40      193.0
3            36       -6.0
4             7       21.0
...         ...        ...
6549          4      error
6550          4      error
6551          4      error
6552          4      error
6553          7      error

[6554 rows x 2 columns]
